## Fluent Python

This is an extract from the book Fluent Python by Luciano Ramalho. The following is an example of the UML class based design pattern. To know more check out chapter 10 of Fluent Python.

In [1]:
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional

In [2]:
class Customer(NamedTuple):
    name: str
    fidelity: int

In [3]:
class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal:
        return self.price * self.quantity

In [4]:
class Order(NamedTuple):
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None
    
    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))
    
    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self) -> str:
        return f"<Order total: {self.total():.2f} due: {self.due():.2f}>"

In [5]:
class Promotion(ABC):
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        pass

In [6]:
class FidelityPromo(Promotion):
    def discount(self, order: Order) -> Decimal:
        return (
            order.total() * Decimal("0.05")
            if order.customer.fidelity >= 1000
            else Decimal(0)
        )

In [7]:
class BulkItemPromo(Promotion):
    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal("0.1")
        return discount

In [8]:
class LargeOrderPromo(Promotion):
    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal("0.07")
        return Decimal(0)

In [9]:
joe = Customer("John Doe", 0)
ann = Customer("Ann Smith", 1100)
cart = [
    LineItem("banana", 4, Decimal("0.5")),
    LineItem("apple", 10, Decimal("1.5")),
    LineItem("watermellon", 5, Decimal("5.0")),
]

In [10]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [11]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [12]:
banana_cart = (
    LineItem("banana", 30, Decimal(".5")),
    LineItem("apple", 10, Decimal("1.5")),
)

In [13]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [14]:
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) for sku in range(10))

In [15]:
Order(joe, long_cart, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [16]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>